# Symbolic Compilation - t|ket> example

Motivation: In compilation, particularly of hybrid classical-quantum variational algorithms in which the structure of a circuit remains constant but the parameters of some gates change, it can be useful to compile using symbolic mathematics and optimise the circuit without knowledge of what these parameters will be instantiated to afterwards.

In this tutorial, we will show how to compile a circuit which has symbols, and then instantiate the symbols afterwards. To do this, you need to have pytket installed. Run:
```
pip install pytket
```
We will also be using the circuit drawing tool from IBM's Qiskit, although this is only for visualisation and is not necessary to do symbolic compilation using pytket. To use the converter:
```
pip install pytket_qiskit
```
To begin the demo, we will import the Circuit and Transform classes from pytket, and the `fresh_symbol` method from pytket._circuit.

In [1]:
from pytket import Circuit, Transform
from pytket._circuit import fresh_symbol

Now, we can construct a circuit containing symbols. You can ask for symbols by calling the `fresh_symbol` method with a string as an argument. This string represents the preferred symbol name - if this is disallowed because it has already been used elsewhere, an appropriate suffix of the form "_x", with 'x' a natural number, will be added to generate a new symbol, as shown below:

In [2]:
a = fresh_symbol("a")
a1 = fresh_symbol("a")
print(a)
print(a1)

a
a_1


We are going to make a circuit made out of just 3 "phase-gadgets" - `Rz` gates surrounded by ladders of `CX` gates.

In [3]:
b = fresh_symbol("b")
circ = Circuit(4)
circ.CX(0,1)
circ.CX(1,2)
circ.CX(2,3)
circ.Rz(3,a)
circ.CX(2,3)
circ.CX(1,2)
circ.CX(0,1)
circ.CX(3,2)
circ.CX(2,1)
circ.CX(1,0)
circ.Rz(0,b)
circ.CX(1,0)
circ.CX(2,1)
circ.CX(3,2)
circ.CX(0,1)
circ.CX(1,2)
circ.CX(2,3)
circ.Rz(3,0.5)
circ.CX(2,3)
circ.CX(1,2)
circ.CX(0,1)

<tket::Circuit qubits=4, gates=21>

Now we can use IBM's Qiskit circuit visualiser to display the circuit. For more explanation of our converters, see the "transform_example" notebook. Note that Qiskit can, conveniently, use symbolics as well.

In [4]:
from pytket.qiskit import tk_to_qiskit

def print_tkcirc_via_qiskit(tkcirc):
    qiskit_qcirc = tk_to_qiskit(tkcirc)
    print(qiskit_qcirc)
    
print_tkcirc_via_qiskit(circ)

                                                       ┌───┐┌──────────┐┌───┐»
q_0: |0>──■────────────────────────────────────■───────┤ X ├┤ Rz(pi*b) ├┤ X ├»
        ┌─┴─┐                                ┌─┴─┐┌───┐└─┬─┘└──────────┘└─┬─┘»
q_1: |0>┤ X ├──■──────────────────────────■──┤ X ├┤ X ├──■────────────────■──»
        └───┘┌─┴─┐                      ┌─┴─┐├───┤└─┬─┘                      »
q_2: |0>─────┤ X ├──■────────────────■──┤ X ├┤ X ├──■────────────────────────»
             └───┘┌─┴─┐┌──────────┐┌─┴─┐└───┘└─┬─┘                           »
q_3: |0>──────────┤ X ├┤ Rz(pi*a) ├┤ X ├───────■─────────────────────────────»
                  └───┘└──────────┘└───┘                                     »
«                                                      
«q_0: ───────■──────────────────────────────────────■──
«     ┌───┐┌─┴─┐                                  ┌─┴─┐
«q_1: ┤ X ├┤ X ├──■────────────────────────────■──┤ X ├
«     └─┬─┘├───┤┌─┴─┐                        ┌─┴─┐└───┘
«q_2: ──■

Now let's use a Transform to shrink the circuit. For more detail on Transforms, see the "transform_example" notebook.

In [5]:
Transform.OptimisePhaseGadgets().apply(circ)
print_tkcirc_via_qiskit(circ)

                                                    
q_0: |0>──■──────────────────────────────────────■──
        ┌─┴─┐                                  ┌─┴─┐
q_1: |0>┤ X ├──■────────────────────────────■──┤ X ├
        ├───┤┌─┴─┐┌──────────────────────┐┌─┴─┐├───┤
q_2: |0>┤ X ├┤ X ├┤ U1(pi*(a + b + 0.5)) ├┤ X ├┤ X ├
        └─┬─┘└───┘└──────────────────────┘└───┘└─┬─┘
q_3: |0>──■──────────────────────────────────────■──
                                                    


Note that the type of gate has changed to a `U1`, but the phase-gadgets have been successfully combined. The `U1` gate is an IBM-specific gate that is equivalent to an `Rz` up to global phase.

We can now instantiate the symbols with some desired values. We make a dictionary, with each key a symbol name, and each value a double. Note that this value is in units of "half-turns", a natural unit in which π = 1.

We will make a circuit copy of the symbolic circuit, so that we can hold this in memory and, each time we need to instantiate our parameters, we can just copy this and instantiate, without requiring compilation again.

In [6]:
symbol_circ = circ.copy()

symbol_dict = {a : 0.5, b : 0.75}
circ.symbol_substitution(symbol_dict)

In [7]:
print_tkcirc_via_qiskit(circ)

                                           
q_0: |0>──■─────────────────────────────■──
        ┌─┴─┐                         ┌─┴─┐
q_1: |0>┤ X ├──■───────────────────■──┤ X ├
        ├───┤┌─┴─┐┌─────────────┐┌─┴─┐├───┤
q_2: |0>┤ X ├┤ X ├┤ U1(1.75*pi) ├┤ X ├┤ X ├
        └─┬─┘└───┘└─────────────┘└───┘└─┬─┘
q_3: |0>──■─────────────────────────────■──
                                           


Because this symbol substitution is local to the circuit it is called on, we still have a symbolic circuit.

In [8]:
print_tkcirc_via_qiskit(symbol_circ)

                                                    
q_0: |0>──■──────────────────────────────────────■──
        ┌─┴─┐                                  ┌─┴─┐
q_1: |0>┤ X ├──■────────────────────────────■──┤ X ├
        ├───┤┌─┴─┐┌──────────────────────┐┌─┴─┐├───┤
q_2: |0>┤ X ├┤ X ├┤ U1(pi*(a + b + 0.5)) ├┤ X ├┤ X ├
        └─┬─┘└───┘└──────────────────────┘└───┘└─┬─┘
q_3: |0>──■──────────────────────────────────────■──
                                                    


Note: the expression tree for this symbolic expression is very small and only consists of a couple of different operations, but t|ket> is capable of large and complicated expressions containing many different types of operation, eg trig functions.

Obviously, it is possible to instantiate a circuit with values such that the circuit can be optimised further - if we had chosen `a=1.5` and `b=0.`, for example, this circuit would be equal to identity. If there are likely to be many parameters set to trivial values (eg 0 or 1), performing further optimisation post-instantiation can be beneficial.